In [ ]:
import pandas as pd
import numpy as np
import os

INPUT_FOLDER = 'Feature_Selected_Data' # 使用特征筛选后的数据
OUTPUT_FOLDER = 'Feature_Engineered_Data_v3' # 新输出文件夹
TRAIN_PRICE_FILE = 'train_price_selected.csv'
TEST_PRICE_FILE = 'test_price_selected.csv'
TRAIN_PRICE_OUT_FILE = 'train_price_eng_v3.csv'
TEST_PRICE_OUT_FILE = 'test_price_eng_v3.csv'

TARGET_COL = 'Price'
AREA_COL = '面积_数值'
BUILD_YEAR_COL = '建筑年份'
TRANSACTION_YEAR_COL = '交易年份'
AGE_COL = '房屋年龄_交易时' # 假设这个列存在且无效值为 -1
GROUPING_COLS_BROADER = ['区域', '区县'] # 更大范围聚合列
GROUPING_COL_PLATE = '板块' # 板块聚合特征已在上一轮加入，这里用来计算单位价格
PLATE_MEAN_PRICE_COL = f'{GROUPING_COL_PLATE}_平均价格'
PLATE_MEAN_AREA_COL = f'{GROUPING_COL_PLATE}_平均面积'

# 设施丰富度相关列 (需要根据你实际筛选后保留的列调整)
FACILITY_COLS = [col for col in pd.read_csv(os.path.join(INPUT_FOLDER, TRAIN_PRICE_FILE), nrows=0).columns 
                 if col.startswith('配套_') or col.startswith('交通_')]
print(f"用于计算设施丰富度的列: {FACILITY_COLS}") # 打印出来确认

epsilon = 1e-6 # 防止除零

# --- 2. 创建输出文件夹 ---
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --- 3. 加载数据 ---
print(f"\n--- 从 '{INPUT_FOLDER}' 加载 Price 数据 ---")
try:
    train_df = pd.read_csv(os.path.join(INPUT_FOLDER, TRAIN_PRICE_FILE), encoding='utf-8-sig')
    test_df = pd.read_csv(os.path.join(INPUT_FOLDER, TEST_PRICE_FILE), encoding='utf-8-sig')
    print("Price 数据加载成功。")
except FileNotFoundError as e:
    print(f"加载数据时出错: {e}. 请确保文件路径正确。")
    exit()
except Exception as e:
    print(f"加载数据时发生其他错误: {e}")
    exit()

# --- 4. 计算二级特征 ---

processed_cols = set() # 记录已处理的列，避免重复添加

# --- a) 房龄平方 ---
print("\n计算 房龄平方...")
if AGE_COL in train_df.columns and AGE_COL in test_df.columns:
    # 先将无效年龄(-1)替换为0，再计算平方
    train_df['房龄平方'] = train_df[AGE_COL].replace(-1, 0) ** 2
    test_df['房龄平方'] = test_df[AGE_COL].replace(-1, 0) ** 2
    processed_cols.add('房龄平方')
    print("  完成。")
else:
    print(f"  跳过：缺少 '{AGE_COL}' 列。")

# --- b) 单位面积价格_板块 ---
print("\n计算 单位面积价格_板块...")
required_cols_usp = [PLATE_MEAN_PRICE_COL, PLATE_MEAN_AREA_COL]
if all(col in train_df.columns for col in required_cols_usp) and \
   all(col in test_df.columns for col in required_cols_usp):
    train_df['单位面积价格_板块'] = train_df[PLATE_MEAN_PRICE_COL] / (train_df[PLATE_MEAN_AREA_COL] + epsilon)
    test_df['单位面积价格_板块'] = test_df[PLATE_MEAN_PRICE_COL] / (test_df[PLATE_MEAN_AREA_COL] + epsilon)
    # 限制在合理范围或处理NaN/inf (如果分母可能为0)
    train_df['单位面积价格_板块'] = np.nan_to_num(train_df['单位面积价格_板块'], nan=0.0, posinf=0.0, neginf=0.0)
    test_df['单位面积价格_板块'] = np.nan_to_num(test_df['单位面积价格_板块'], nan=0.0, posinf=0.0, neginf=0.0)
    processed_cols.add('单位面积价格_板块')
    print("  完成。")
else:
    print(f"  跳过：缺少板块聚合特征列 ({required_cols_usp})。")

# --- c) 装修与房龄交互 ---
print("\n计算 装修与房龄交互...")
required_cols_reno_age = [AGE_COL, '装修_精装', '装修_毛坯'] # 假设精装和毛坯的哑变量存在
if all(col in train_df.columns for col in required_cols_reno_age) and \
   all(col in test_df.columns for col in required_cols_reno_age):
    # 处理年龄为-1的情况，将交互结果设为0
    age_train = train_df[AGE_COL].replace(-1, 0)
    age_test = test_df[AGE_COL].replace(-1, 0)
    
    train_df['装修_精装_x_房龄'] = train_df['装修_精装'] * age_train
    test_df['装修_精装_x_房龄'] = test_df['装修_精装'] * age_test
    
    train_df['装修_毛坯_x_房龄'] = train_df['装修_毛坯'] * age_train
    test_df['装修_毛坯_x_房龄'] = test_df['装修_毛坯'] * age_test
    
    processed_cols.add('装修_精装_x_房龄')
    processed_cols.add('装修_毛坯_x_房龄')
    print("  完成。")
else:
    print(f"  跳过：缺少 '{AGE_COL}' 或 '装修_精装'/'装修_毛坯' 列。")

# --- d) 更大范围的区域聚合特征 ---
print("\n计算 更大范围的区域聚合特征...")
# 先计算全局统计量
global_mean_price = train_df[TARGET_COL].mean()
global_median_price = train_df[TARGET_COL].median()
global_mean_area = train_df[AREA_COL].mean()

for group_col in GROUPING_COLS_BROADER:
    print(f"  处理分组: '{group_col}'")
    required_cols_agg_broad = [group_col, TARGET_COL, AREA_COL]
    if not all(col in train_df.columns for col in required_cols_agg_broad) or \
       group_col not in test_df.columns:
        print(f"    跳过：缺少聚合所需的列或分组列 '{group_col}'。")
        continue

    # 计算统计量 (训练集)
    agg_stats = train_df.groupby(group_col).agg(
        **{f'{group_col}_平均价格': pd.NamedAgg(column=TARGET_COL, aggfunc='mean'),
           f'{group_col}_中位数价格': pd.NamedAgg(column=TARGET_COL, aggfunc='median'),
           f'{group_col}_平均面积': pd.NamedAgg(column=AREA_COL, aggfunc='mean')}
    ).reset_index()

    # 合并
    try:
        train_df = pd.merge(train_df, agg_stats, on=group_col, how='left')
        test_df = pd.merge(test_df, agg_stats, on=group_col, how='left')
    except Exception as e:
         print(f"    合并 '{group_col}' 聚合特征时出错: {e}")
         continue # 跳过填充和记录

    # 填充 NaN
    fillna_cols = [f'{group_col}_平均价格', f'{group_col}_中位数价格', f'{group_col}_平均面积']
    fill_values = {
        f'{group_col}_平均价格': global_mean_price,
        f'{group_col}_中位数价格': global_median_price,
        f'{group_col}_平均面积': global_mean_area
    }
    test_df.fillna(fill_values, inplace=True)
    train_df.fillna(fill_values, inplace=True) # 也填充训练集（如果原始 group_col 有 NaN）
    
    # 记录新列名
    for col in fillna_cols: processed_cols.add(col)
    print(f"    完成 '{group_col}' 聚合特征。")

# --- e) 卫浴比 ---
print("\n计算 卫浴比...")
required_cols_bath = ['户型_卫', '户型_室']
if all(col in train_df.columns for col in required_cols_bath) and \
   all(col in test_df.columns for col in required_cols_bath):
    train_df['卫浴比'] = train_df['户型_卫'] / (train_df['户型_室'] + epsilon)
    test_df['卫浴比'] = test_df['户型_卫'] / (test_df['户型_室'] + epsilon)
    # 处理分母为0的情况 (例如 0室)
    train_df.loc[train_df['户型_室'] <= 0, '卫浴比'] = 0
    test_df.loc[test_df['户型_室'] <= 0, '卫浴比'] = 0
    processed_cols.add('卫浴比')
    print("  完成。")
else:
    print(f"  跳过：缺少 '{' 或 '.join(required_cols_bath)}' 列。")

# --- f) 设施丰富度 ---
print("\n计算 设施丰富度...")
# 再次检查 FACILITY_COLS 是否真的在 DataFrame 中
actual_facility_cols_train = [col for col in FACILITY_COLS if col in train_df.columns]
actual_facility_cols_test = [col for col in FACILITY_COLS if col in test_df.columns]

if actual_facility_cols_train and actual_facility_cols_test:
    # 确保两边列一致（理论上应该一致）
    if set(actual_facility_cols_train) == set(actual_facility_cols_test):
        train_df['设施丰富度'] = train_df[actual_facility_cols_train].sum(axis=1)
        test_df['设施丰富度'] = test_df[actual_facility_cols_test].sum(axis=1)
        processed_cols.add('设施丰富度')
        print(f"  完成 (基于 {len(actual_facility_cols_train)} 个特征)。")
    else:
        print("  跳过：训练集和测试集的设施列不完全匹配。")
elif not FACILITY_COLS:
     print("  跳过：未定义用于计算丰富度的设施列。")
else:
     print("  跳过：数据中缺少定义的设施列。")


print(f"\n二级特征工程完成。新增/修改了 {len(processed_cols)} 个特征。")

# --- 5. 保存添加了新特征的数据 ---
print(f"\n--- 保存最终特征工程后的数据到 '{OUTPUT_FOLDER}' ---")
try:
    train_output_path = os.path.join(OUTPUT_FOLDER, TRAIN_PRICE_OUT_FILE)
    test_output_path = os.path.join(OUTPUT_FOLDER, TEST_PRICE_OUT_FILE)
    
    train_df.to_csv(train_output_path, index=False, encoding='utf-8-sig')
    test_df.to_csv(test_output_path, index=False, encoding='utf-8-sig')
    
    print(f"✓ 处理后的训练集已保存: {train_output_path} (新形状: {train_df.shape})")
    print(f"✓ 处理后的测试集已保存: {test_output_path} (新形状: {test_df.shape})")
    
except Exception as e:
    print(f"保存文件时出错: {e}")

print("\n--- 脚本执行完毕 ---")

# --- (可选) 验证 ---
print("\n--- 验证新特征 (训练集部分示例) ---")
new_cols_to_show = list(processed_cols)
if new_cols_to_show:
     print(train_df[new_cols_to_show[:min(len(new_cols_to_show), 10)]].head()) # 最多显示10个新列
     print("\n新特征统计描述:")
     print(train_df[new_cols_to_show].describe())
else:
     print("没有成功创建新特征进行验证。")

开始执行二级特征工程...
用于计算设施丰富度的列: ['配套_医院', '配套_公园', '配套_银行', '配套_商场', '配套_市场', '配套_幼儿园', '配套_购物广场', '配套_社区', '配套_休闲', '配套_医疗', '配套_百货', '配套_菜场', '配套_三甲', '配套_体育', '配套_酒店', '配套_步行街', '配套_便利店', '配套_森林公园', '交通_地铁', '交通_公交', '交通_公交站', '交通_高速', '交通_机场', '交通_自驾', '交通_大道', '交通_轻轨', '交通_隧道', '交通_快速路', '交通_高架', '交通_高铁', '交通_枢纽站']

--- 从 'Feature_Selected_Data' 加载 Price 数据 ---
Price 数据加载成功。

计算 房龄平方...
  跳过：缺少 '房屋年龄_交易时' 列。

计算 单位面积价格_板块...
  跳过：缺少板块聚合特征列 (['板块_平均价格', '板块_平均面积'])。

计算 装修与房龄交互...
  跳过：缺少 '房屋年龄_交易时' 或 '装修_精装'/'装修_毛坯' 列。

计算 更大范围的区域聚合特征...
  处理分组: '区域'
    完成 '区域' 聚合特征。
  处理分组: '区县'
    完成 '区县' 聚合特征。

计算 卫浴比...
  完成。

计算 设施丰富度...
  完成 (基于 31 个特征)。

二级特征工程完成。新增/修改了 8 个特征。

--- 保存最终特征工程后的数据到 'Feature_Engineered_Data_v3' ---
✓ 处理后的训练集已保存: Feature_Engineered_Data_v3\train_price_eng_v3.csv (新形状: (103871, 163))
✓ 处理后的测试集已保存: Feature_Engineered_Data_v3\test_price_eng_v3.csv (新形状: (34017, 163))

--- 脚本执行完毕 ---

--- 验证新特征 (训练集部分示例) ---
        卫浴比      区县_中位数价格     区县_平均面积      区域_中位数价格       区域_